In [ ]:
# Taken from https://github.com/robotallie/mlb-disabled-list/blob/master/datasets/injury_data_scrape.ipynb
# Altered to assess our project's specific needs.

# We used the final data set obtained from this scraper, to then join onto
# our other full MLB database and continue our data cleaning process.

# Data from this scraper was taken from https://www.prosportstransactions.com
# MLB Disabled List data

# Import and Install Packages needed for Web-Scraper.
import csv
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
df = pd.DataFrame()
# Get a list of dictionaries for the injuries 
injuries_data = []
for i in range(0,657): # number of page increments plus one
    url_string = "http://www.prosportstransactions.com/baseball/Search/SearchResults.php?Player=&Team=&BeginDate=2000-03-01&EndDate=2016-10-31&DLChkBx=yes&submit=Search&start="+str(25* i )
    req = requests.get(url_string)
    soup = BeautifulSoup(req.content, 'lxml')
    for item in soup.find_all("tr", {"align":"left"}):# Code for each individual page to capture data
        raw_text = item.text.strip().split("\n")
        injuries_data.append(raw_text)

# Create a dataframe from the injuries data for 723 pages, with 25 per page = 18075 ish        
df = pd.DataFrame(injuries_data)
df.head()

,0,1,2,3,4
0,2000-03-23,Rockies,,• John Thomson,placed on 60-day DL recovering from surgery o...
1,2000-04-01,Devil Rays,,• Wilson Alvarez,placed on DL
2,2000-04-01,Dodgers,,• Antonio Osuna,placed on 15-day DL
3,2000-04-08,Mets,,• Darryl Hamilton,placed on 15-day DL with sprained left foot
4,2000-04-08,Orioles,,• Mike Timlin,placed on 15-day DL with strained abdominal


In [ ]:
df.columns = ['Date','Team','Acquired','Name','Notes']
df.head()

,Date,Team,Acquired,Name,Notes
0,2000-03-23,Rockies,,• John Thomson,placed on 60-day DL recovering from surgery o...
1,2000-04-01,Devil Rays,,• Wilson Alvarez,placed on DL
2,2000-04-01,Dodgers,,• Antonio Osuna,placed on 15-day DL
3,2000-04-08,Mets,,• Darryl Hamilton,placed on 15-day DL with sprained left foot
4,2000-04-08,Orioles,,• Mike Timlin,placed on 15-day DL with strained abdominal


In [ ]:
df.shape

(16425, 5)

In [ ]:
# Let's make a copy to keep the original web scraped data safe
injury_df = pd.DataFrame(df).copy()

In [ ]:
# The Acquired and Name columns don't have nulls but they should. 
# They have ' '
injury_df.Acquired = injury_df.Acquired.apply(lambda x: np.nan if x == ' ' else x.replace('•','').split(' / ')[0])

In [ ]:
# Get rid of the bullet point and second names for the player names for what used to be the relinquished col
injury_df.Name = injury_df.Name.apply(lambda x: np.nan if x == ' ' else x.replace('•','').split(' / ')[0])

In [ ]:
def filter_notes_for_DL(notes):
    """Function parses notes column for number
    of days player was relinquished to the DL.
    Returns the number, 7, 10, 15, 60 or 0."""
    if '15' in notes:
        return 15
    elif '60' in notes:
        return 60
    elif '10' in notes:
        return 10
    elif '7' in notes:
        return 7
    elif 'restricted' in notes:
        return 0
    elif 'temporary' in notes:
        return 0
    else:
        return 0
    
# Create a column that lists the length the player was on the list DL
injury_df['DL_length'] = injury_df.Notes.map(filter_notes_for_DL)

In [ ]:
injury_df.Name.isnull().sum()
# 7115 

# Create a dummy column that is 1 if the row represents an injury 
# or a 0 if the row represents a player reactivated.
injury_df['Injured'] = injury_df.Name.map(lambda x: 1 if pd.notnull(x) else 0)
# Value Counts for Injury:
# 1    9288
# 0    7115

In [ ]:
# Start to extract the number out of the Notes column.
# Replace the hyphen in '15-day' with a space to help splitting and extracting digits.
# injury_df.Notes = injury_df.Notes.apply(lambda x: x.replace('-',' '))

def extract_injury(notes):
    """Function parses notes column
    to obtain the injury type and returns a string"""
    if len(notes.split('DL')) >= 2:
        if len(notes.split('with')) > 1:
            return notes.split('with')[1]
        elif len(notes.split('recovering from')) > 1:
            return notes.split('recovering from')[1]
        else:
            if notes.split('DL')[1] == '':
                return "unknown"
            else:
                return notes.split('DL')[1]
    else:
        return 'unknown'

injury_df.Notes.head(20)

0      placed on 60-day DL recovering from surgery o...
1                                          placed on DL
2                                   placed on 15-day DL
3           placed on 15-day DL with sprained left foot
4           placed on 15-day DL with strained abdominal
5               placed on 15-day DL with strained groin
6                              activated from 15-day DL
7                                   placed on 15-day DL
8                              activated from 15-day DL
9                                   placed on 15-day DL
10                                  placed on 15-day DL
11                             activated from 15-day DL
12                                  placed on 15-day DL
13                         activated from inactive list
14                                  placed on 15-day DL
15                                  placed on 15-day DL
16                                  placed on 15-day DL
17                                  placed on 15

In [ ]:
# Create a column that describes the type of injury based on the notes column using
# the function I created: extract_injury, df['Injury_Type']
injury_df['Injury_Type'] = df.Notes.map(extract_injury)
injury_df['Injury_Type'].head()

0     surgery on shoulder
1                 unknown
2                 unknown
3      sprained left foot
4      strained abdominal
Name: Injury_Type, dtype: object

In [ ]:
# What kind of injuries are we looking at? 
# 49% of the DL list placements did not have a description of the injury, and the rest account for less than 2%
# of all injuries (because the variation in description prevents cataloging)
injury_df['Injury_Type'].value_counts(normalize=True)

unknown                                                                  0.489680
 strained left hamstring                                                 0.015647
 right shoulder inflammation                                             0.012481
 strained right hamstring                                                0.012298
 strained right shoulder                                                 0.009619
                                                                           ...   
 surgery to correct anal fissure                                         0.000061
 surgery on right hamstring                                              0.000061
 surgery on left elbow (transferred from 60-day to 15-day DL per MLB)    0.000061
 torn lateral meniscus in left knee                                      0.000061
 herniated disc in back (out for season)                                 0.000061
Name: Injury_Type, Length: 2286, dtype: float64

In [ ]:
# Let's change the date to match the baseball gamelogs:
injury_df['Date'] = injury_df['Date'].map(lambda x: x.replace('-',''))
injury_df['Date'].head()

0    20000323
1    20000401
2    20000401
3    20000408
4    20000408
Name: Date, dtype: object

In [ ]:
from google.colab import drive
drive.mount('drive')
df.to_csv('MLB-DL-Data.csv')
!cp MLB-DL-Data.csv "drive/My Drive/"

Mounted at drive


In [ ]:
file = 'drive/My Drive/TeamCurrentNames.csv'
team_slugs_df = pd.read_csv(file,header=None)
team_slugs_df.columns = ['old_slug','new_slug','league', 
                         'division','location','nickname','alt_nick','date_start','date_end','city','state']
# Cut the dataframe down to slugs that are recent since 2000 or still current
team_slugs_df['date_end'] = pd.to_datetime(team_slugs_df['date_end'])
team_slugs_df = team_slugs_df[(team_slugs_df['date_end'] > '03/01/2000') | (team_slugs_df['date_end'].isnull())]

team_slugs_df.head()

team_slugs_mini = team_slugs_df[['new_slug','nickname']]
team_slugs_mini.set_index('nickname', inplace=True)

# Turn team_slug_df into key:value pairs
team_slugs_dict = team_slugs_mini.to_dict()['new_slug']

In [ ]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

injury_df['team_id'] = injury_df['Team'].apply(lambda x: team_slugs_dict[process.extract(x, team_slugs_dict.keys(), limit=1)[0][0]])
injury_df.columns

# Select only the columns we need for the game log matching (except keep Injured for the next step alone)
cols = ['Date', 'Team','Name', 'DL_length','Injured', 'team_id']
df_injury_sm = injury_df[cols]

# Remove rows where df_injury_sm['Injury']==0 because we only want injuries, not activations
print('Before removing reactivations:',df_injury_sm.shape)
df_injury_sm = df_injury_sm[df_injury_sm.Injured != 0]
print('With only placements onto the Disabled List:',df_injury_sm.shape)

# Now get rid of the Injured column
df_injury_sm.drop('Injured', inplace=True, axis=1)
df_injury_sm.head()

Before removing reactivations: (16425, 6)
With only placements onto the Disabled List: (9329, 6)


,Date,Team,Name,DL_length,team_id
0,20000323,Rockies,John Thomson,60,COL
1,20000401,Devil Rays,Wilson Alvarez,0,TBA
2,20000401,Dodgers,Antonio Osuna,15,LAN
3,20000408,Mets,Darryl Hamilton,15,NYN
4,20000408,Orioles,Mike Timlin,15,BAL


In [ ]:
# Add player id from retrosheet.org info
file = 'drive/My Drive/MLBPlayerIDs.txt'
ids = pd.read_csv(file)
print(ids.columns)

# The file rows are wrapped in "", so I'll strip the rows into columns the long way:
ids.columns
player_ids = pd.DataFrame(columns=['ID', 'Last', 'First', 'Player_debut', 'Mgr_debut', 'Coach_debut','Ump_debut'])
player_ids['ID'] = ids.ID.apply(lambda x: x.split(',')[0])
player_ids['Last'] = ids.ID.apply(lambda x: x.split(',')[1].strip('"'))
player_ids['First'] = ids.ID.apply(lambda x: x.split(',')[2].strip('"'))
player_ids['Player_debut'] = ids.ID.apply(lambda x: x.split(',')[3])
player_ids['Mgr_debut'] = ids.ID.apply(lambda x: x.split(',')[4])
player_ids['Coach_debut'] = ids.ID.apply(lambda x: x.split(',')[5])
player_ids['Ump_debut'] = ids.ID.apply(lambda x: x.split(',')[6])
player_ids.drop(['Mgr_debut','Coach_debut','Ump_debut'],axis=1,inplace=True)
player_ids.head()
player_ids.to_csv('player_ids.csv',index=False)
# The entire database of players, managers and umpires holds 20,493 unique ids. I don't need the 
# guys who played in the 1800s, sadly.
print('Before cutting out the old guys',player_ids.shape)

# Reduce the table down to those players who debuted after 1975 (who would have been 25 year veterans )

# Turn the Player_debut column into a datetime object rather than a string:
player_ids.Player_debut = pd.to_datetime(player_ids.Player_debut, infer_datetime_format=True)

# Select debut dates after 1975-03-15
player_ids = player_ids[player_ids.Player_debut > '1975-03-15']

# After reducing the dataframe to just post 1975 debuts, there are less than 8000 rows left. Reduced by half.
print('After cutting out the old guys',player_ids.shape)
# Merge the injury_df with the player_ids is complex because the injury_df has full name that may or may not match
player_ids['Full_Name'] = player_ids.apply(lambda x: x.loc['First'] +' ' +x.loc['Last'], axis=1)

# There seems to be a problem with the names of the injury df having whitespace characters around them:
df_injury_sm.Name = df_injury_sm.Name.apply(lambda x: x.strip())

# Try to assign the id by row
# for row in df_injury_sm:
#     df_injury_sm['player_id'] = 
player_id_list = []
# Iterate through each player injury in the injury database, 
for index, row in df_injury_sm.iterrows():
    if np.sum(player_ids['Full_Name'] == row['Name']) >= 1:
        player_id_list.append(player_ids[player_ids['Full_Name'] == row['Name']]['ID'].iloc[0,])
    else:
        player_id_list.append(np.nan)

# I ended up with 1273 null values, which means for 1273 players, there wasn't a perfect match in the list.
# Go back later and try fuzzywuzzy.
player_id_df = pd.DataFrame(player_id_list, columns=['player_id'])

# Check to make sure the player_id column has the same number of rows as the injury_df before adding it:
print('Player_id dataframe shape')
print(player_id_df.shape)
print('Injury dataframe shape')
print(df_injury_sm.shape)
# Now that the full name is in the player_ids dataframe, I will try to merge this id into my injury_df
df_injury_sm['player_id'] = player_id_df.values

# Check columns and values:
print(player_id_df.describe())

print(df_injury_sm['player_id'].describe())

# Note: What's interesting is there are only 2373 unique player ids out of 8015. Multiple visits by the same 
# player -- but that's strange.
df_injury_sm.head()
# All I need is the date, the dl_length, the player_id for this.
final_injury_df = df_injury_sm[['Date','DL_length','player_id']].copy()
final_injury_df.head()
# The format of the Date column needs to be updated, since that was lost along the way:
final_injury_df['Date'] = final_injury_df['Date'].map(lambda x: x.replace('-',''))
final_injury_df.head()
final_injury_df['DL_length'].median()
# 15.0

final_injury_df['DL_length'] = final_injury_df['DL_length'].replace(0,15)

Index(['ID', 'Last', 'First', 'Player debut', 'Mgr debut', 'Coach debut',
       'Ump debut'],
      dtype='object')
Before cutting out the old guys (20493, 4)
After cutting out the old guys (7966, 4)
Player_id dataframe shape
(9329, 1)
Injury dataframe shape
(9329, 6)
       player_id
count       7763
unique      2305
top     beckj002
freq          21
count         7763
unique        2305
top       beckj002
freq            21
Name: player_id, dtype: object


In [ ]:
df_injury_sm.head()

,Date,Team,Name,DL_length,team_id,player_id
0,20000323,Rockies,John Thomson,60,COL,thomj005
1,20000401,Devil Rays,Wilson Alvarez,0,TBA,alvaw001
2,20000401,Dodgers,Antonio Osuna,15,LAN,osuna002
3,20000408,Mets,Darryl Hamilton,15,NYN,hamid001
4,20000408,Orioles,Mike Timlin,15,BAL,timlm001


In [ ]:
df_injury_sm.to_csv('df_injury_sm.csv')
!cp df_injury_sm.csv "drive/My Drive/"